# AFET Visualization Demo
This notebook demonstrates the visualization capabilities of the AI Fairness and Explainability Toolkit (AFET).

In [ ]:
# Install required packages if not already installed
!pip install plotly pandas scikit-learn numpy

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Import AFET visualization components
from afet.viz import (
    FairnessDashboard,
    ThresholdAnalysis,
    plot_confusion_matrices
)

## 1. Generate Sample Data
Create a synthetic dataset with known biases for demonstration.

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

# Generate synthetic data
n_samples = 2000

# Generate sensitive attribute (e.g., gender)
sensitive = np.random.choice(['Male', 'Female'], size=n_samples, p=[0.6, 0.4])

# Create synthetic features with some bias
n_features = 10
X = np.random.randn(n_samples, n_features)

# Add bias to the sensitive attribute
X[sensitive == 'Male', 0] += 0.5  # Feature 0 is more predictive for males
X[sensitive == 'Female', 1] += 0.5  # Feature 1 is more predictive for females

# Generate target with some bias
coef = np.random.randn(n_features)
coef[0] = 1.5  # Make feature 0 more important
coef[1] = 1.2  # Make feature 1 more important

# Add bias based on sensitive attribute
sensitive_effect = np.where(sensitive == 'Male', 0.3, -0.3)
y_prob = 1 / (1 + np.exp(-(X @ coef + sensitive_effect + np.random.randn(n_samples) * 0.5)))
y = (y_prob > 0.5).astype(int)

# Split into train and test sets
X_train, X_test, y_train, y_test, sensitive_train, sensitive_test = train_test_split(
    X, y, sensitive, test_size=0.3, random_state=42, stratify=y
)

# Train a model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Get predictions
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

print(f"Dataset size: {len(X)}")
print(f"Positive class ratio: {y.mean():.2f}")
print(f"Sensitive attribute distribution:\n{pd.Series(sensitive).value_counts(normalize=True).to_string()}")

## 2. Fairness Dashboard
Create an interactive dashboard to analyze model fairness.

In [ ]:
# Initialize dashboard
dashboard = FairnessDashboard(
    model_names=['RandomForest'],
    sensitive_attr='Gender'
)

# Create dashboard
fig = dashboard.create_dashboard(
    y_true=y_test,
    y_pred={'RandomForest': y_pred},
    y_prob={'RandomForest': y_prob},
    sensitive=sensitive_test,
    feature_names=[f'feature_{i}' for i in range(n_features)],
    feature_importances={'RandomForest': model.feature_importances_}
)

# Show dashboard
fig.show()

## 3. Threshold Analysis
Analyze the trade-off between model performance and fairness across different classification thresholds.

In [ ]:
# Create threshold analysis
fig = ThresholdAnalysis.plot_threshold_analysis(
    y_true=y_test,
    y_prob=y_prob,
    sensitive=sensitive_test,
    model_name='RandomForest'
)

# Show figure
fig.show()

## 4. Confusion Matrices
Compare confusion matrices across different models or sensitive groups.

In [ ]:
# Train a second model for comparison
from sklearn.linear_model import LogisticRegression

model2 = LogisticRegression(max_iter=1000, random_state=42)
model2.fit(X_train, y_train)
y_pred2 = model2.predict(X_test)

# Create confusion matrices
fig = plot_confusion_matrices(
    y_true=y_test,
    y_pred_dict={
        'RandomForest': y_pred,
        'LogisticRegression': y_pred2
    },
    model_names=['RandomForest', 'LogisticRegression'],
    class_names=['Rejected', 'Approved']
)

# Show figure
fig.show()

## 5. Analyzing Model Biases
Compare model performance and fairness metrics across sensitive groups.

In [ ]:
from afet.core.fairness_metrics import FairnessMetrics
import pandas as pd

# Calculate metrics for each model
metrics = []

for name, preds in [('RandomForest', y_pred), ('LogisticRegression', y_pred2)]:
    fm = FairnessMetrics(
        y_true=y_test,
        y_pred=preds,
        sensitive_features=sensitive_test
    )
    
    metrics.append({
        'Model': name,
        'Accuracy': fm.accuracy(),
        'Precision': fm.precision(),
        'Recall': fm.recall(),
        'F1': fm.f1_score(),
        'Demographic Parity': fm.demographic_parity_difference(),
        'Equal Opportunity': fm.equal_opportunity_difference(),
        'Average Odds': fm.average_odds_difference()
    })

# Create a DataFrame with the results
results_df = pd.DataFrame(metrics)
results_df.set_index('Model', inplace=True)

# Display the results
results_df.style
    .format('{:.3f}')
    .background_gradient(cmap='viridis', axis=0)
    .set_caption('Model Performance and Fairness Metrics')

## 6. Saving Visualizations
Save the visualizations for reports or presentations.

In [ ]:
import os

# Create output directory
os.makedirs('output', exist_ok=True)

# Save dashboard
dashboard_fig = dashboard.create_dashboard(
    y_true=y_test,
    y_pred={'RandomForest': y_pred},
    y_prob={'RandomForest': y_prob},
    sensitive=sensitive_test,
    feature_names=[f'feature_{i}' for i in range(n_features)],
    feature_importances={'RandomForest': model.feature_importances_}
)
dashboard_fig.write_html('output/fairness_dashboard.html')

# Save threshold analysis
threshold_fig = ThresholdAnalysis.plot_threshold_analysis(
    y_true=y_test,
    y_prob=y_prob,
    sensitive=sensitive_test,
    model_name='RandomForest'
)
threshold_fig.write_html('output/threshold_analysis.html')

# Save confusion matrices
confusion_fig = plot_confusion_matrices(
    y_true=y_test,
    y_pred_dict={
        'RandomForest': y_pred,
        'LogisticRegression': y_pred2
    },
    model_names=['RandomForest', 'LogisticRegression'],
    class_names=['Rejected', 'Approved']
)
confusion_fig.write_html('output/confusion_matrices.html')

print('Visualizations saved to the output directory.')

## 7. Conclusion
This notebook demonstrated the visualization capabilities of the AFET library, including:
- Interactive fairness dashboards
- Threshold analysis for fairness-accuracy trade-offs
- Confusion matrix comparison
- Comprehensive model evaluation

These tools can help identify and mitigate biases in machine learning models, leading to fairer and more transparent AI systems.